In [1]:
#Hate speech recognition
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
print(tf.__version__)

2.5.0


In [3]:
import os

import numpy as np

# For DataFrame object
import pandas as pd

# Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

# Text Vectorizing
from keras.preprocessing.text import Tokenizer

# Train-test-split
from sklearn.model_selection import train_test_split

# History visualization
%matplotlib inline
import matplotlib.pyplot as plt

# Normalize
from sklearn.preprocessing import normalize

In [ ]:
path = 'labeled.csv' #Need to add labeled hate speech texts!
df = pd.read_csv(path)
df.head()

In [ ]:
def delete_new_line_symbols(text):
    text = text.replace('\n', ' ')
    return text
df['comment'] = df['comment'].apply(delete_new_line_symbols)
df.head()

In [ ]:
target = np.array(df['toxic'].astype('uint8'))
target[:5]

In [ ]:
tokenizer = Tokenizer(num_words=30000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
                      lower=True, 
                      split=' ', 
                      char_level=False)
tokenizer.fit_on_texts(df['comment'])
matrix = tokenizer.texts_to_matrix(df['comment'], mode='count')
matrix.shape

In [ ]:
def get_model():
    
    model = Sequential()
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=RMSprop(lr=0.0001), 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
X = normalize(matrix)
y = target

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.2)

X_train.shape, y_train.shape

In [ ]:
model = get_model()

history = model.fit(X_train, 
                    y_train, 
                    epochs=150, 
                    batch_size=500,
                    validation_data=(X_test, y_test))

history

In [ ]:
history = history.history

fig = plt.figure(figsize=(20, 10))

ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(223)

x = range(150)

ax1.plot(x, history['acc'], 'b-', label='Accuracy')
ax1.plot(x, history['val_acc'], 'r-', label='Validation accuracy')
ax1.legend(loc='lower right')

ax2.plot(x, history['loss'], 'b-', label='Losses')
ax2.plot(x, history['val_loss'], 'r-', label='Validation losses')
ax2.legend(loc='upper right')